In [ ]:
options(jupyter.plot_mimetypes = c("text/plain", "image/png"))
options(jupyter.plot_scale = 1.5)
require("pacman")
pacman::p_load(tidyverse, lubridate, zoo, ggpubr, ggfortify, forecast)

source('../Scripts/ReadData.R')
source('../Scripts/ModelSelection.R')
source('../Scripts/ResidualAnalysis.R')
source('../Scripts/Forecasting.R')

In [ ]:
homeownership <- read_homeownership()
homeownership <- homeownership %>% mutate(dValue = Value - lag(Value)) %>% drop_na

p1 <- homeownership %>% ggplot() +
    geom_line(aes(x = Date, y = dValue)) +
    labs(y = "Change in Homeownership Rate", title = "Day over Day Change in Homeownership Rate")
p2 <- ggAcf(homeownership$dValue) + 
    labs(title = "ACF for Change in Homeownership Rate")
ggarrange(p1, p2, nrow = 2, ncol = 1)

In [ ]:
train <- head(homeownership, -4)
test <- tail(homeownership, 4)
arima_fit <- fit_arima(training_data = train$dValue, p_range = 0:5, d_range = 0:2, q_range = 0:5)
head(arima_fit)

In [ ]:
suppressWarnings(arima_model <- arima(train$dValue, order = c(4,0,4), method = "ML"))
residual_analysis(arima_model, train$Date)
box_tests(arima_model, 8)

In [ ]:
arma_garch_fit <- fit_ugarch(train$dValue, 4, 4, 0:5, 0:5, 0:2, 0:2)
head(arma_garch_fit$Final)

In [ ]:
spec <- ugarchspec(variance.model=list(garchOrder=c(0,1)),
                     mean.model=list(armaOrder=c(3,2),
                                     include.mean=T), distribution.model="std")
suppressWarnings(ugarch_model1 <- ugarchfit(spec, train$dValue, solver = 'hybrid'))
residual_analysis(ugarch_model1, train$Date)
box_tests(ugarch_model1, 6)

In [ ]:
spec <- ugarchspec(variance.model=list(garchOrder=c(2,0)),
                     mean.model=list(armaOrder=c(3,2),
                                     include.mean=T), distribution.model="std")
suppressWarnings(ugarch_model2 <- ugarchfit(spec, train$dValue, solver = 'hybrid'))
residual_analysis(ugarch_model2, train$Date)
box_tests(ugarch_model2, 7)

In [ ]:
spec <- ugarchspec(variance.model=list(garchOrder=c(1,1)),
                     mean.model=list(armaOrder=c(3,2),
                                     include.mean=T), distribution.model="std")
suppressWarnings(ugarch_model3 <- ugarchfit(spec, train$dValue, solver = 'hybrid'))
residual_analysis(ugarch_model3, train$Date)
box_tests(ugarch_model3, 8)

In [ ]:
fc <- forecast_arima(arima_model, homeownership$Date, test$dValue, 4, 95,
                    y_lab = "Change in Homeownership Rate (%)")
fc$plot
fc_data <- tail(fc$data, 4)
cat("PM = ", prec_measure(fc_data$Observed, fc_data$Forecast))
accuracy(fc_data$Forecast, x = fc_data$Observed)

In [ ]:
fc <- forecast_ugarchroll(ugarch_model1, homeownership$Date, test$dValue, 4,
                          y_lab = c("Change in Homeownership Rate (%)", "Change in Homeownership Rate Variance"))
fc$plot
fc_data <- tail(fc$data, 4)
cat("PM = ", prec_measure(fc_data$Observed, fc_data$Forecast))
accuracy(fc_data$Forecast, x = fc_data$Observed)

In [ ]:
fc <- forecast_ugarchroll(ugarch_model2, homeownership$Date, test$dValue, 4,
                          y_lab = c("Change in Homeownership Rate (%)", "Change in Homeownership Rate Variance"))
fc$plot
fc_data <- tail(fc$data, 4)
cat("PM = ", prec_measure(fc_data$Observed, fc_data$Forecast))
accuracy(fc_data$Forecast, x = fc_data$Observed)

In [ ]:
fc <- forecast_ugarchroll(ugarch_model3, homeownership$Date, test$dValue, 4,
                          y_lab = c("Change in Homeownership Rate (%)", "Change in Homeownership Rate Variance"))
fc$plot
fc_data <- tail(fc$data, 4)
cat("PM = ", prec_measure(fc_data$Observed, fc_data$Forecast))
accuracy(fc_data$Forecast, x = fc_data$Observed)